In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

The following code uses python to produce a pandas DataFrame with columns for Match ID ("id"), Innings ("inning"), Batting Position ("position"), and Batsman ("batsman").  

VERSION 3 NOTE:  Version 3 is just Version 2 with these notes added.  Version 1 was based on an analysis of batsmen, but only of batsmen.  This may have allowed inaccuracies to creep in.  Examples:
* Batsmen Agarwal and Bairstow are the first wicket partnership, and are 1 and 2 in the batting order, respectively.  Agarwal is caught out, but Agarwal and Bairstow crossed, meaning that the 3rd batsmen in the batting order, de Villiers, enters the game at the non-striker's end.  On the next delivery, Bairstow is dismissed LBW, and is replaced by the 4th batsman, Finch.  Version 1 would have indicated Finch at position 3, then de Villiers at position 4 as Finch would have batted before de Villiers.
    
* In the event that a batsman ever entered the game as the non-striker but never batted, they would have been left out of the analysis, as they would not have been registered in the batsman column of the DataFrame

In [ ]:
deliveries = pd.read_csv("/kaggle/input/ipl-complete-dataset-20082020/IPL Ball-by-Ball 2008-2020.csv")


Re-sort data (original data set was off-set, such that initial row is ball 5 of over 6 from the first match)

In [ ]:
deliveries = deliveries.sort_values(by=["id", "inning", "over", "ball"]).reset_index()
deliveries = deliveries.drop(columns = ["index"])
deliveries

Establish list to fill future data frame. Will be a list of lists, with 4 elements per internal list: id, inning, position batsman

In [ ]:
temp = [[deliveries.iloc[0][0], deliveries.iloc[0][1], 1, deliveries.iloc[0][4]], [deliveries.iloc[0][0], deliveries.iloc[0][1], 2, deliveries.iloc[0][5]]]
temp

Evaluate all subsequent rows of deliveries to determine if either batsman or non-striker is new

In [ ]:
#for loop: check all rows from 1-> end
i = 1
for i in range(1, len(deliveries)):
#rule for first row of match
    if deliveries.iloc[i][0] != deliveries.iloc[(i-1)][0]:
        temp.append([deliveries.iloc[i][0], deliveries.iloc[i][1], 1, deliveries.iloc[i][4]]) 
        temp.append([deliveries.iloc[i][0], deliveries.iloc[i][1], 2, deliveries.iloc[i][5]])
#rule for first row of new innings
    elif deliveries.iloc[i][1] != deliveries.iloc[(i-1)][1]:
        temp.append([deliveries.iloc[i][0], deliveries.iloc[i][1], 1, deliveries.iloc[i][4]]) 
        temp.append([deliveries.iloc[i][0], deliveries.iloc[i][1], 2, deliveries.iloc[i][5]])
#rule for other rows
    elif deliveries.iloc[i][4] != deliveries.iloc[(i-1)][4] and deliveries.iloc[i][4] != deliveries.iloc[(i-1)][5]:
        temp.append([deliveries.iloc[i][0], deliveries.iloc[i][1], (temp[-1][2] + 1), deliveries.iloc[i][4]])
    elif deliveries.iloc[i][5] != deliveries.iloc[(i-1)][4] and deliveries.iloc[i][5] != deliveries.iloc[(i-1)][5]:
        temp.append([deliveries.iloc[i][0], deliveries.iloc[i][1], (temp[-1][2] + 1), deliveries.iloc[i][5]])

Turn list into dataframe

In [ ]:
batsmen_with_positions = pd.DataFrame(temp, columns = ['id', 'inning', 'position', 'batsman'])

View data for verification

In [ ]:
batsmen_with_positions